In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import AnalysisFunctions as afuns
from functools import reduce
import os

In [ ]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [ ]:
def get_bdata_filename(exp_path):
    bdata_filename = [filename for filename in os.listdir(exp_path+sep+r'with food\blob analysis normalized by white paper') 
                        if filename.startswith(r'bdata')][0]
    return bdata_filename

In [ ]:
def plot_event(ax,start_frame, end_frame, color, alpha):
    ax.axvspan(start_frame, end_frame, facecolor=color, alpha=alpha, zorder=-100)

In [ ]:
def get_ant_data(ant, bdata, crops, tdata, transparency, dt=10, fdata=False, conversion_factors=False):
    ant_data={}
    
    ant_data['ant'] = ant
    
    raw_x = bdata['a'+str(ant)+'-x']
    raw_y = bdata['a'+str(ant)+'-y']
    ant_data['x']=raw_x.interpolate(method='linear',limit_area='inside')
    ant_data['y']=raw_y.interpolate(method='linear',limit_area='inside')
    
    ant_data['v']=calc_vel(ant_data['x'],ant_data['y'],dt)
    
    ant_data['transp'] = transparency.transparency[transparency['ant']==ant].iloc[0]
    
    raw_crop = bdata['a'+str(ant)+'-crop_intensity']/ant_data['transp']/ant_data['transp']
    
    crop_df_raw = pd.DataFrame({'frame':raw_crop.index, 'raw_crop':raw_crop.values, 'acquisition':bdata['acquisition'].values})
    crop_df_clean = pd.DataFrame({'clean_red':crops[('red',str(ant))], 'clean_yellow':crops[('yellow',str(ant))]})
    crop_df = crop_df_raw.merge(crop_df_clean,left_index=True, right_index=True)
    
    if isinstance(conversion_factors, pd.DataFrame):
        crop_df['raw_crop'][crop_df['acquisition']=='BLGF']=crop_df['raw_crop'][crop_df['acquisition']=='BLGF']/conversion_factors['yellow'][0]
        crop_df['raw_crop'][crop_df['acquisition']=='GLRF']=crop_df['raw_crop'][crop_df['acquisition']=='GLRF']/conversion_factors['red'][0]
        crop_df['clean_red']=crop_df['clean_red']/conversion_factors['red'][0]
        crop_df['clean_yellow']=crop_df['clean_yellow']/conversion_factors['yellow'][0]
        
    ant_data['crop_df']=crop_df
    
    ant_data['interactions']=tdata[(tdata['giver']==ant) | (tdata['receiver']==ant)]
    
    if isinstance(fdata, pd.DataFrame):
        ant_data['feedings'] = fdata[fdata.ant_id==ant]
    
    return ant_data


def calc_vel(x,y,dt):
    diff_x = x.diff(periods=dt)
    diff_y = y.diff(periods=dt)
    v = np.sqrt(diff_x**2 + diff_y**2)/dt
    return v

In [ ]:
def get_intervals_between_events(ant_data):
    interactions = pd.DataFrame()
    interactions['start_frame'] = ant_data['interactions']['start_frame']
    interactions['end_frame'] = ant_data['interactions']['end_frame']
    interactions['event_type'] = 'interaction'

    feedings = pd.DataFrame()
    feedings['start_frame'] = ant_data['feedings']['feeding_start']
    feedings['end_frame'] = ant_data['feedings']['feeding_end']
    feedings['event_type'] = 'feeding'

    all_events = pd.concat([interactions, feedings]).sort_values(by='start_frame')
    all_events_df = all_events.copy()

    interval_starts=[0]
    interval_ends=[]
    while len(all_events)>0:

        next_event = all_events.iloc[0]
        next_event_start = next_event['start_frame']
        next_events = all_events[all_events['start_frame']<=next_event['end_frame']]  # to deal with overlapping events
        next_event_end = max(next_events['end_frame'])  # if next events are overlapping - take the end of the one that ends last
        
        interval_ends.append(next_event_start)
        interval_starts.append(next_event_end+1)
        
        all_events = update_remaining_events(interval_starts[-1], all_events)

    interval_ends.append(bdata['frame'].iloc[-1])
    
    return interval_starts, interval_ends, all_events_df
    

def update_remaining_events(frame, events):
    events=events[events['start_frame']>=frame]
    return events

In [ ]:
# Classify interactions to unloading, receiving and small
def classify_interactions(behavior_df, ant_data, min_volume):
    giving_df = ant_data['interactions'][ant_data['interactions']['giver']==ant]
    unloading = giving_df[['start_frame','end_frame']][(giving_df['transferred_red']>=min_volume) | (giving_df['transferred_yellow']>=min_volume)]
    small_giving = giving_df[['start_frame','end_frame']][(giving_df['transferred_red']<min_volume) | (giving_df['transferred_yellow']<min_volume)]
    
    receiving_df = ant_data['interactions'][ant_data['interactions']['receiver']==ant]
    receiving = receiving_df[['start_frame','end_frame']][(receiving_df['transferred_red']>=min_volume) | (receiving_df['transferred_yellow']>=min_volume)]
    small_receiving = receiving_df[['start_frame','end_frame']][(receiving_df['transferred_red']<min_volume) | (receiving_df['transferred_yellow']<min_volume)]
    
    small_interactions = pd.concat([small_giving, small_receiving])
    
    for df, behavior in zip([unloading, receiving, small_interactions], ['non-receiving_interaction','receiving_interaction','non-receiving_interaction']):
        df['behavior'] = behavior
    
    behavior_df = pd.concat([behavior_df, unloading, receiving, small_interactions])
    behavior_df = behavior_df.sort_values(by='start_frame').reset_index(drop=True)
    
    return behavior_df

In [ ]:
# insert feedings to behavior_df
def insert_feedings(behavior_df, ant_data):
    feedings_df = ant_data['feedings'][['feeding_start','feeding_end','food_source']]
    feedings_df=feedings_df.rename(columns={'feeding_start':'start_frame', 'feeding_end':'end_frame','food_source':'behavior'})
    feedings_df['behavior']=feedings_df['behavior'].apply(lambda x: x + '_feeding')
    
    behavior_df = pd.concat([behavior_df, feedings_df])
    behavior_df = behavior_df.sort_values(by='start_frame').reset_index(drop=True)
    
    return behavior_df

In [ ]:
def merge_consecutive_events(behavior_df, window_size):
    behavior_df['next_behavior']=behavior_df['behavior'].shift(-1)
    behavior_df['next_start']=behavior_df['start_frame'].shift(-1)
    behavior_df['next_end']=behavior_df['end_frame'].shift(-1)
    
    group_labels=[]
    counter = 1
    chunk_ends = []
    in_chunk = False
    for idx, event in behavior_df.iterrows():
        if not to_merge(event, window_size):
            group_labels.append(0)
            counter += 1
            if in_chunk:
                chunk_ends.append(idx)
            in_chunk=False
        else:
            group_labels.append(counter)
            in_chunk=True
    
    behavior_df['group_labels']=group_labels
    
    groups = np.unique(group_labels)
    groups = groups[groups>0]
    
    merged_events=[]
    for g in groups:
        chunk = behavior_df[behavior_df['group_labels']==g]
        merged_event = chunk.iloc[0][['start_frame','end_frame','behavior']]
        merged_event['end_frame']=chunk.iloc[-1]['next_end']
        merged_events.append(merged_event)
    
    merged_df = pd.concat(merged_events,axis=1).transpose()
    solitary_events = behavior_df[behavior_df['group_labels']==0]
    solitary_events=solitary_events.drop(index=chunk_ends)
    behavior_df = pd.concat([merged_df,solitary_events[['start_frame','end_frame','behavior']]]).sort_values(by='start_frame')
    behavior_df=behavior_df.reset_index(drop=True)
    
    return behavior_df

def to_merge(event, window_size):
        m = (event['behavior']==event['next_behavior']) & ((event['next_start']-event['end_frame'])<=window_size)
        return m

In [ ]:
def is_unloading(merged_event, crop_df, min_vol):
    red_change = crop_df['clean_red'][merged_event['start_frame']]-crop_df['clean_red'][merged_event['end_frame']]
    yellow_change = crop_df['clean_yellow'][merged_event['start_frame']]-crop_df['clean_yellow'][merged_event['end_frame']]
    res = (red_change>min_vol) | (yellow_change>min_vol)
    return res

In [ ]:
exp_num = 11
exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)
bdata_filename = get_bdata_filename(exp_path)

bdata = pd.read_csv(exp_path + sep + r'with food\blob analysis normalized by white paper' + sep + bdata_filename, index_col=0)
fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
conversion_factors = pd.read_csv(exp_path + sep + r'conversion_factors_by_weight_and_feeding_sum.csv')
transparency = pd.read_csv(exp_path + sep + r'transparency_table.csv')

In [ ]:
min_volume = 1  # ul
dt = 10  # frames
min_move = 50  # pixels/dt frames

smoothing_window = 200 # frames